Patient 1 

In [11]:
import mne
import json
import torch
mne.set_log_level('ERROR')

Patient 1

In [1]:
CHB_files1 = {
    'CHB-MIT/CHB_Database/chb01/chb01_01.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_02.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_03.edf': [(0, 2396, 'interictal'), (2396, 2996, 'preictal'), (2996, 3036, 'ictal'), (3036, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_04.edf': [(0, 867, 'interictal'), (867, 1467, 'preictal'), (1467, 1494, 'ictal'), (1494, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_05.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_06.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_07.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_08.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_09.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_10.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_11.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_12.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_13.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_14.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_15.edf': [(0, 1132, 'interictal'), (1132, 1732, 'preictal'), (1732, 1772, 'ictal'), (1772, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_16.edf': [(0, 415, 'interictal'), (415, 1015, 'preictal'), (1015, 1066, 'ictal'), (1066, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_17.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_18.edf': [(0, 1120, 'interictal'), (1120, 1720, 'preictal'), (1720, 1810, 'ictal'), (1810, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_19.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_20.edf': [(0, 2663, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_21.edf': [(0, 327, 'preictal'), (327, 420, 'ictal'), (420, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_22.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_23.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_24.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_25.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_26.edf': [(0, 1262, 'interictal'), (1262, 1862, 'preictal'), (1862, 1963, 'ictal'), (1963, 2325, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_27.edf': [(0, 600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_29.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_30.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_31.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_32.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_33.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_34.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_36.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_37.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_38.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_39.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_40.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_41.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_42.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_43.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_46.edf': [(0, 3600, 'interictal')]
    }

In [12]:
def segment_data(filepath, info, segment_length, segment_dict, dict_length):
    raw = mne.io.read_raw_edf(filepath, preload=True)  # Load the EDF file
    eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True)  # Select EEG channels
    freqs = (60, 120)  # Define notch filter frequencies to remove power line noise
    raw.notch_filter(freqs=freqs, picks=eeg_picks)  # Apply notch filter
    raw.filter(l_freq=30, h_freq=None, fir_design='firwin', filter_length='auto', phase='zero', fir_window='hamming')
    raw.set_meas_date(None)  # Reset measurement date

    for segment_info in info:
        start, end, label = segment_info
        current_position = start
        while current_position < end:
            tmin = current_position
            tmax = min(current_position + segment_length, end)  # Ensure tmax does not exceed the label's end
            segment_filename = f"{dict_length}-chb02.pt"
            segment_path = f"CHB-MIT/Segments/chb02/{segment_filename}"
            
            raw_segment = raw.copy().crop(tmin=tmin, tmax=tmax, include_tmax=False)
            numpy_array = raw_segment.get_data()
       
            
            segment_dict[segment_path] = label
            
            segment_tensor = torch.from_numpy(numpy_array).unsqueeze(0)
            torch.save(segment_tensor,segment_path)
            dict_length += 1  # Increment for a unique filename
            current_position += segment_length  # Move to the next segment start

    return dict_length

In [16]:
CHB_files2 = {
    'CHB-MIT/CHB_Database/chb02/chb02_01.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_02.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_03.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_04.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_05.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_06.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_07.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_08.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_09.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_10.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_11.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_12.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_13.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_14.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_15.edf': [(0, 3130, 'interictal'), (3130, 3600, 'preictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_16.edf': [(0,130, 'preictal'), (130, 212, 'ictal'), (212, 959, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_16+.edf': [(0, 2372, 'interictal'),(2372, 2972, 'preictal'),(2972, 3053, 'ictal'), (3053, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_17.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_18.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_19.edf': [(0, 2769, 'interictal'), (2769, 3369, 'preictal'), (3369, 3378, 'ictal'), (3378, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_20.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_21.edf': [(0, 327, 'preictal'), (327, 420, 'ictal'), (420, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_22.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_23.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_24.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_25.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_26.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_27.edf': [(0, 600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_29.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_30.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_31.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_32.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_33.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_34.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb02/chb02_35.edf': [(0, 3600, 'interictal')],
    }

In [13]:
def write_dict_to_json_file(segment_dict, filepath):
    with open(filepath, 'w') as file:
        json.dump(segment_dict, file)
        

In [17]:
segment_dict = {}
dict_length = 0  # Initialize dict_length to 0
segment_length = 1  # Define the length of each segment to be 1 second

# Adapted example usage to process each file in CHB_files with its associated segment info
for file_path, segment_info in CHB_files2.items():
    # Now, segment_data processes all segments for a given file in one call
    # The function itself handles iteration over the segments defined in segment_info
    dict_length = segment_data(file_path, segment_info, segment_length, segment_dict, dict_length)

# After processing all files, save the segment dictionary to a JSON file
write_dict_to_json_file(segment_dict, 'CHB-MIT/segment_dict2.json')

Patient 2

In [ ]:
CHB_files3 = {
    'CHB-MIT/CHB_Database/chb03/chb03_01.edf': [(0, 362, 'preictal'), (362, 414, 'ictal'), (414, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_02.edf': [(0, 131, 'interictal'), (131, 731, 'preictal'), (731, 796, 'ictal'), (796, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_03.edf': [(0, 432, 'preictal'), (432, 501, 'ictal'), (501, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_04.edf': [(0, 1562, 'interictal'), (1562, 2162, 'preictal'), (2162, 2214, 'ictal'), (2214, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_05.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_06.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_07.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_08.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_09.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_10.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_11.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_12.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_13.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_14.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_15.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_16.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_17.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_18.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_19.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_20.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_21.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_22.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_23.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_24.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_25.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_26.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_27.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_28.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_29.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_30.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_31.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_32.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_33.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_34.edf': [(0, 1382, 'interictal'), (1382, 1982, 'preictal'), (1982, 2029, 'ictal'), (2029, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_35.edf': [(0, 1992, 'interictal'), (1992, 2592, 'preictal'), (2592, 2656, 'ictal'), (2656, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_36.edf': [(0, 1125, 'interictal'), (1125, 1725, 'preictal'), (1725, 1778, 'ictal'), (1778, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_37.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb03/chb03_38.edf': [(0, 3600, 'interictal')]
    }

In [1]:
CHB_files4 = {
'chb04_01.edf': [(0, 14398, 'interictal')],
'chb04_02.edf': [(0, 14400, 'interictal')],
'chb04_03.edf': [(0, 14400, 'interictal')],
'chb04_04.edf': [(0, 14400, 'interictal')],
'chb04_05.edf': [
    (0, 7204, 'interictal'),
    (7204, 7804, 'preictal'),
    (7804, 7853, 'ictal'),
    (7853, 9536, 'interictal')
],
'chb04_06.edf': [(0, 14400, 'interictal')],
'chb04_07.edf': [(0, 7496, 'interictal')],
'chb04_08.edf': [
    (0, 5846, 'interictal'),
    (5846, 6446, 'preictal'),
    (6446, 6557, 'ictal'),
    (6557, 14400, 'interictal')
],
'chb04_09.edf': [(0, 14400, 'interictal')],
'chb04_10.edf': [(0, 14400, 'interictal')],
'chb04_11.edf': [(0, 4832, 'interictal')],
'chb04_12.edf': [(0, 14420, 'interictal')],
'chb04_13.edf': [(0, 14398, 'interictal')],
'chb04_14.edf': [(0, 14418, 'interictal')],
'chb04_15.edf': [(0, 14400, 'interictal')],
'chb04_16.edf': [(0, 14400, 'interictal')],
'chb04_17.edf': [(0, 13732, 'interictal')],
'chb04_18.edf': [(0, 14425, 'interictal')],
'chb04_19.edf': [(0, 14400, 'interictal')],
'chb04_21.edf': [(0, 14400, 'interictal')],
'chb04_22.edf': [(0, 14400, 'interictal')],
'chb04_23.edf': [(0, 10684, 'interictal')],
'chb04_24.edf': [(0, 14406, 'interictal')],
'chb04_25.edf': [(0, 14400, 'interictal')],
'chb04_26.edf': [(0, 14400, 'interictal')],
'chb04_27.edf': [(0, 14427, 'interictal')],
'chb04_28.edf': [
    (0, 1079, 'interictal'),
    (1079, 1679, 'preictal'),
    (1679, 1781, 'ictal'),
    (1781, 3182, 'interictal'),
    (3182, 3782, 'preictal'),
    (3782, 3898, 'ictal'),
    (3898, 14423, 'interictal')
],
'chb04_29.edf': [(0, 14403, 'interictal')], 'chb04_30.edf': [(0, 14400, 'interictal')],
'chb04_31.edf': [(0, 14400, 'interictal')],
'chb04_32.edf': [(0, 14400, 'interictal')],
'chb04_33.edf': [(0, 14400, 'interictal')],
'chb04_34.edf': [(0, 14400, 'interictal')],
'chb04_35.edf': [(0, 14400, 'interictal')],
'chb04_36.edf': [(0, 14400, 'interictal')],
'chb04_37.edf': [(0, 5613, 'interictal')],
'chb04_38.edf': [(0, 14400, 'interictal')],
'chb04_39.edf': [(0, 14400, 'interictal')],
'chb04_40.edf': [(0, 14400, 'interictal')],
'chb04_41.edf': [(0, 14400, 'interictal')],
'chb04_42.edf': [(0, 5823, 'interictal')],
'chb04_43.edf': [(0, 14400, 'interictal')]
}

In [ ]:
CHB_files5 = {
'chb05_01.edf': [(0, 3610, 'interictal')],
'chb05_02.edf': [(0, 3600, 'interictal')],
'chb05_03.edf': [(0, 3600, 'interictal')],
'chb05_04.edf': [(0, 3600, 'interictal')],
'chb05_05.edf': [(0, 3600, 'interictal')],
'chb05_06.edf': [(0, 3600, 'interictal'),
(0, 417, 'preictal'),
(417, 532, 'ictal'),
(532, 3600, 'interictal')],
'chb05_07.edf': [(0, 3600, 'interictal')],
'chb05_08.edf': [(0, 3600, 'interictal')],
'chb05_09.edf': [(0, 3600, 'interictal')],
'chb05_10.edf': [(0, 3600, 'interictal')],
'chb05_11.edf': [(0, 3600, 'interictal')],
'chb05_12.edf': [(0, 3600, 'interictal')],
'chb05_13.edf': [(0, 3600, 'interictal'),
(486, 1086, 'preictal'),
(1086, 1196, 'ictal'),
(1196, 3600, 'interictal')],
'chb05_14.edf': [(0, 3600, 'interictal')],
'chb05_15.edf': [(0, 3600, 'interictal')],
'chb05_16.edf': [(0, 3600, 'interictal'),
(1717, 2317, 'preictal'),
(2317, 2413, 'ictal'),
(2413, 3600, 'interictal')],
 'chb05_17.edf': [(0, 2451, 'interictal'), (2451, 2571, 'ictal'), (2571, 3600, 'interictal')],
 'chb05_18.edf': [(0, 3600, 'interictal')],
 'chb05_19.edf': [(0, 3600, 'interictal')],
 'chb05_20.edf': [(0, 3600, 'interictal')],
 'chb05_21.edf': [(0, 3600, 'interictal')],
 'chb05_22.edf': [(0, 2348, 'interictal'), (2348, 2465, 'ictal'), (2465, 3600, 'interictal')],
 'chb05_23.edf': [(0, 3600, 'interictal')],
 'chb05_24.edf': [(0, 3600, 'interictal')],
 'chb05_25.edf': [(0, 3600, 'interictal')],
 'chb05_26.edf': [(0, 3600, 'interictal')],
 'chb05_27.edf': [(0, 3600, 'interictal')],
 'chb05_28.edf': [(0, 3600, 'interictal')],
 'chb05_29.edf': [(0, 3600, 'interictal')],
 'chb05_30.edf': [(0, 3600, 'interictal')],
 'chb05_31.edf': [(0, 3600, 'interictal')],
 'chb05_32.edf': [(0, 3600, 'interictal')],
 'chb05_33.edf': [(0, 3600, 'interictal')],
 'chb05_34.edf': [(0, 3600, 'interictal')],
 'chb05_35.edf': [(0, 3600, 'interictal')],
 'chb05_36.edf': [(0, 3600, 'interictal')],
 'chb05_37.edf': [(0, 3600, 'interictal')],
 'chb05_38.edf': [(0, 3600, 'interictal')],
 'chb05_39.edf': [(0, 3600, 'interictal')],
}

In [ ]:
CHB_files6 = {
'chb06_01.edf': [(0, 14427, 'interictal'),
(1124, 1724, 'preictal'),
(1724, 1738, 'ictal'),
(1738, 6861, 'interictal'),
(6861, 7461, 'preictal'),
(7461, 7476, 'ictal'),
(7476, 12925, 'interictal'),
(12925, 13525, 'preictal'),
(13525, 13540, 'ictal'),
(13540, 14427, 'interictal')],
'chb06_02.edf': [(0, 14400, 'interictal')],
'chb06_03.edf': [(0, 14400, 'interictal')],
'chb06_04.edf': [(0, 13261, 'interictal'),
(0, 327, 'preictal'),
(327, 347, 'ictal'),
(347, 5611, 'interictal'),
(5611, 6211, 'preictal'),
(6211, 6231, 'ictal'),
(6231, 13261, 'interictal')],
'chb06_05.edf': [(0, 14400, 'interictal')],
'chb06_06.edf': [(0, 14400, 'interictal')],
'chb06_07.edf': [(0, 14400, 'interictal')],
'chb06_08.edf': [(0, 14400, 'interictal')],
'chb06_09.edf': [(0, 14400, 'interictal'),
(11900, 12500, 'preictal'),
(12500, 12516, 'ictal'),
(12516, 14400, 'interictal')],
'chb06_10.edf': [(0, 14400, 'interictal'),
(10233, 10833, 'preictal'),
(10833, 10845, 'ictal'),
(10845, 14400, 'interictal')],
'chb06_12.edf': [(0, 14400, 'interictal')],
'chb06_13.edf': [(0, 14400, 'interictal'),
(0, 506, 'preictal'),
(506, 519, 'ictal'),
(519, 14400, 'interictal')],
'chb06_14.edf': [(0, 14400, 'interictal')],
'chb06_15.edf': [(0, 14400, 'interictal')],
'chb06_16.edf': [(0, 3030, 'interictal')],
'chb06_17.edf': [(0, 14400, 'interictal')],
'chb06_18.edf': [(0, 7928, 'interictal'),
(7199, 7799, 'preictal'),
(7799, 7811, 'ictal'),
(7811, 7928, 'interictal')],
'chb06_24.edf': [(0, 14400, 'interictal'),
(8787, 9387, 'preictal'),
(9387, 9403, 'ictal'),
(9403, 14400, 'interictal')]
 }

In [ ]:
CHB_files7 = {
'chb07_01.edf': [(0, 14411, 'interictal')],
'chb07_02.edf': [(0, 14400, 'interictal')],
'chb07_03.edf': [(0, 14400, 'interictal')],
'chb07_04.edf': [(0, 14400, 'interictal')],
'chb07_05.edf': [(0, 14400, 'interictal')],
'chb07_06.edf': [(0, 4920, 'interictal')],
'chb07_07.edf': [(0, 4660, 'interictal')],
'chb07_08.edf': [(0, 14400, 'interictal')],
'chb07_09.edf': [(0, 14400, 'interictal')],
'chb07_10.edf': [(0, 14400, 'interictal')],
'chb07_11.edf': [(0, 14400, 'interictal')],
'chb07_12.edf': [(0, 4620, 'interictal'),
(4620, 4920, 'preictal'),
(4920, 5006, 'ictal'),
(5006, 14400.0, 'interictal')],
'chb07_13.edf': [(0, 2985, 'interictal'),
(2985, 3285, 'preictal'),
(3285, 3381, 'ictal'),
(3381, 3726, 'interictal')],
'chb07_14.edf': [(0, 14400, 'interictal')],
'chb07_15.edf': [(0, 14400, 'interictal')],
'chb07_16.edf': [(0, 14400, 'interictal')],
'chb07_17.edf': [(0, 14400, 'interictal')],
'chb07_18.edf': [(0, 12060, 'interictal')],
'chb07_19.edf': [(0, 13388, 'interictal'),
(13388, 13688, 'preictal'),
(13688, 13831, 'ictal'),
(13831, 14411, 'interictal')]
 }


In [ ]:
CHB_files8 = {'chb08_02.edf': [(2070, 2670, 'preictal'), (2670, 2841, 'ictal')],
 'chb08_03.edf': [(0, 3600, 'interictal')],
 'chb08_04.edf': [(0, 3600, 'interictal')],
 'chb08_05.edf': [(2256, 2856, 'preictal'), (2856, 3046, 'ictal')],
 'chb08_10.edf': [(0, 3600, 'interictal')],
 'chb08_11.edf': [(2388, 2988, 'preictal'), (2988, 3122, 'ictal')],
 'chb08_12.edf': [(0, 3600, 'interictal')],
 'chb08_13.edf': [(1817, 2417, 'preictal'), (2417, 2577, 'ictal')],
 'chb08_14.edf': [(0, 3600, 'interictal')],
 'chb08_15.edf': [(0, 3600, 'interictal')],
 'chb08_16.edf': [(0, 3600, 'interictal')],
 'chb08_17.edf': [(0, 3600, 'interictal')],
 'chb08_18.edf': [(0, 3600, 'interictal')],
 'chb08_19.edf': [(0, 3600, 'interictal')],
 'chb08_20.edf': [(0, 3600, 'interictal')],
 'chb08_21.edf': [(1483, 2083, 'preictal'), (2083, 2347, 'ictal')],
 'chb08_22.edf': [(0, 3600, 'interictal')],
 'chb08_23.edf': [(0, 3600, 'interictal')],
 'chb08_24.edf': [(0, 3600, 'interictal')],
 'chb08_29.edf': [(0, 3623, 'interictal')]}

In [ ]:
CHB_files9 = {'chb09_01.edf': [(0, 8023.0, 'interictal')],
 'chb09_02.edf': [(0, 14399.0, 'interictal')],
 'chb09_03.edf': [(0, 14400.0, 'interictal')],
 'chb09_04.edf': [(0, 14415.0, 'interictal')],
 'chb09_05.edf': [(0, 14400.0, 'interictal')],
 'chb09_06.edf': [(0, 11331, 'interictal'),
  (11331, 12231, 'preictal'),
  (12231, 12295, 'ictal'),
  (12295, 14399.0, 'interictal')],
 'chb09_07.edf': [(0, 14400.0, 'interictal')],
 'chb09_08.edf': [(0, 2051, 'interictal'),
  (2051, 2951, 'preictal'),
  (2951, 3030, 'ictal'),
  (3030, 8896, 'interictal'),
  (8296, 9196, 'preictal'),
  (9196, 9267, 'ictal'),
  (9267, 14400.0, 'interictal')],
 'chb09_09.edf': [(0, 14400.0, 'interictal')],
 'chb09_10.edf': [(0, 14400.0, 'interictal')],
 'chb09_11.edf': [(0, 14400.0, 'interictal')],
 'chb09_12.edf': [(0, 14400.0, 'interictal')],
 'chb09_13.edf': [(0, 14405.0, 'interictal')],
 'chb09_14.edf': [(0, 14411.0, 'interictal')],
 'chb09_15.edf': [(0, 14421.0, 'interictal')],
 'chb09_16.edf': [(0, 13591.0, 'interictal')],
 'chb09_17.edf': [(0, 974.0, 'interictal')],
 'chb09_18.edf': [(0, 14400.0, 'interictal')],
 'chb09_19.edf': [(0, 4399, 'interictal'),
  (4399, 5299, 'preictal'),
  (5299, 5361, 'ictal'),
  (5361, 5700.0, 'interictal')]}

In [ ]:
CHB_files10 = {'chb10_01.edf': [(0, 7220.0, 'interictal')],
 'chb10_02.edf': [(0, 7200.0, 'interictal')],
 'chb10_03.edf': [(0, 7200.0, 'interictal')],
 'chb10_04.edf': [(0, 7200, 'interictal')],
 'chb10_05.edf': [(0, 7200.0, 'interictal')],
 'chb10_06.edf': [(0, 7200.0, 'interictal')],
 'chb10_07.edf': [(0, 7200.0, 'interictal')],
 'chb10_08.edf': [(0, 7200.0, 'interictal')],
 'chb10_12.edf': [(0, 5713, 'interictal'),
  (5713, 6313, 'preictal'),
  (6313, 6348, 'ictal'),
  (6348, 7200.0, 'interictal')],
 'chb10_13.edf': [(0, 7200.0, 'interictal')],
 'chb10_14.edf': [(0, 7200.0, 'interictal')],
 'chb10_15.edf': [(0, 7200.0, 'interictal')],
 'chb10_16.edf': [(0, 7200.0, 'interictal')],
 'chb10_17.edf': [(0, 7200.0, 'interictal')],
 'chb10_18.edf': [(0, 7200.0, 'interictal')],
 'chb10_19.edf': [(0, 7200.0, 'interictal')],
 'chb10_20.edf': [(0, 6288, 'interictal'),
  (6288, 6888, 'preictal'),
  (6888, 6958, 'ictal'),
  (6958, 7212.0, 'interictal')],
 'chb10_21.edf': [(0, 7200, 'interictal')],
 'chb10_22.edf': [(0, 7200, 'interictal')],
 'chb10_27.edf': [(1482, 2382, 'preictal'), (2382, 2447, 'ictal')],
 'chb10_28.edf': [(0, 7200, 'interictal')],
 'chb10_30.edf': [(2121, 3021, 'preictal'), (3021, 3079, 'ictal')],
 'chb10_31.edf': [(2901, 3801, 'preictal'), (3801, 3877, 'ictal')],
 'chb10_38.edf': [(3718, 4618, 'preictal'), (4618, 4707, 'ictal')],
 'chb10_89.edf': [(483, 1383, 'preictal'), (1383, 1437, 'ictal')]
}